<a href="https://colab.research.google.com/github/Apple03244/DataAnalysis-ML/blob/main/003_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# colab에서 진행했으므로 주석처리

# import google.colab as colab
# f=colab.files

In [6]:
#files=f.upload()

In [7]:
# import pandas as pd
# train=pd.read_csv('/content/train.csv')
# test=pd.read_csv('/content/test.csv')
# submission=pd.read_csv('/content/sample_submission.csv')
# datainfo=pd.read_csv('/content/data_info.csv')

In [8]:
import pandas as pd
train=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\공부파일들\데이콘\학습플랫폼갱신\train.csv")
test=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\공부파일들\데이콘\학습플랫폼갱신\test.csv")

## 데이터 출처
데이콘
이탈고객 예측

### step.1 예측모델을 먼저 만들자

In [9]:
import sklearn.impute as skimp
import sklearn.pipeline as skpip
import sklearn.model_selection as skmod
import sklearn.compose as skcom
import sklearn.metrics as skmet
import sklearn.preprocessing as skpre
import sklearn.discriminant_analysis as skdis
import numpy as np
np.random.seed(10)
import scipy as sp

In [10]:
use_train=train.drop('user_id',axis=1)

In [11]:
num_cols=use_train.select_dtypes(np.number).columns.difference(["target"])
cate_cols=use_train.select_dtypes("object").columns
target="target"

In [12]:
use_train.describe()

,subscription_duration,recent_login_time,average_login_time,average_time_per_learning_session,monthly_active_learning_days,total_completed_courses,recent_learning_achievement,abandoned_learning_sessions,community_engagement_level,customer_inquiry_history,payment_pattern,target
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,11.897400,15.013200,14.994076,54.917720,12.545400,12.227500,75.029513,3.043600,3.886100,2.010500,3.502900,0.619900
std,6.600896,8.362573,3.001869,56.024310,6.932239,3.634125,9.968529,1.755052,1.262175,1.420983,2.311261,0.485435
min,1.000000,1.000000,2.366189,0.011515,1.000000,1.000000,35.941755,0.000000,1.000000,0.000000,0.000000,0.000000
25%,6.000000,8.000000,13.025597,15.276611,7.000000,10.000000,68.278054,2.000000,3.000000,1.000000,1.000000,0.000000
50%,12.000000,15.000000,14.979228,37.578818,13.000000,12.000000,75.126061,3.000000,4.000000,2.000000,4.000000,1.000000
75%,18.000000,22.000000,16.995340,75.584200,19.000000,15.000000,81.718976,4.000000,5.000000,3.000000,6.000000,1.000000
max,23.000000,29.000000,26.998490,503.372616,24.000000,27.000000,112.643828,12.000000,5.000000,10.000000,7.000000,1.000000


In [13]:
# 우선 nan값이 없으므로
num_pip=skpip.make_pipeline(skpre.MinMaxScaler())
cate_pip=skpip.make_pipeline(skpre.OneHotEncoder(handle_unknown='ignore'))
column_trans=skcom.make_column_transformer((num_pip,num_cols),(cate_pip,cate_cols))

# 모델
LD_model=skdis.LinearDiscriminantAnalysis(store_covariance=True)

Total_process=skpip.make_pipeline(column_trans,LD_model)

### step2. 교차검증

In [14]:
val_results=skmod.cross_val_score(estimator=Total_process,X=use_train,y=use_train[target],scoring="accuracy",cv=3,n_jobs=-1)

In [15]:
val_results

array([0.62027594, 0.61986199, 0.61986199])

정확도 60%는 어떤 의미가 있는지 확인해보자  
아래 `dummy_model`은 항상 단순한 값은(아래의 경우 최빈값을 뱉어낼것이다) 추출하는 분류 모델이다

In [16]:
import sklearn.dummy as skdum
dummy_model=skdum.DummyClassifier()
dummy_model.fit(X=use_train[use_train.columns.difference([target])],y=use_train[target])

DummyClassifier()

In [17]:
print(dummy_model.predict(use_train[use_train.columns.difference([target])]))

[1 1 1 ... 1 1 1]


In [18]:
dummy_results=skmod.cross_val_score(estimator=dummy_model,X=use_train[use_train.columns.difference([target])],
                                    y=use_train[target],cv=3,n_jobs=-1)

In [19]:
dummy_results

array([0.619976  , 0.61986199, 0.61986199])

### step3. 해석
위와 같이 정확도 60%는 굉장히 의미없는 숫자임을 우리는 알수있다.  
`dummy_model`을 통해 모두 최빈값으로 예측해도 60%의 정확도를 기대할 수 있기 때문

### step4. confusion matirx
오차 행렬을 살펴보자

In [20]:
predict_matrix=skmod.cross_val_predict(estimator=Total_process,X=use_train,y=use_train[target],cv=3,n_jobs=-1)

In [21]:
skmet.confusion_matrix(y_pred=predict_matrix,y_true=use_train[target])

array([[   2, 3799],
       [   1, 6198]], dtype=int64)

|구조||
|--|--|
|행|실제|
|열|예측|


In [22]:
pd.DataFrame(predict_matrix)[0].value_counts()

0
1    9997
0       3
Name: count, dtype: int64

### 재현율, 정밀도의 필요성

|수치|설명|
|--|--|
|정밀도|y_pred==True 중 y_true==True값의 비중|
|재현율|y_true==True 중 y_pred==True값의 비중|
|f1-score|둘의 조화평균|

### 오차 행렬
|구분|pred=0|pred=1|
|--|--|--|
|**true=0**|True_Negative|False_Positive|
|**true=1**|False_Negative|True_Positive|

### 참고.직접 계산해보면서 알아보자

1. 정확도

In [31]:
confu_mtx=skmet.confusion_matrix(y_pred=predict_matrix,y_true=use_train[target])
(confu_mtx[0,0]+confu_mtx[1,1])/np.sum(confu_mtx)

0.62

In [30]:
skmet.accuracy_score(y_true=use_train[target],y_pred=predict_matrix)

0.62

2. 정밀도

In [33]:
confu_mtx[1,1]/(confu_mtx[0,1]+confu_mtx[1,1])

0.6199859957987396

In [34]:
skmet.precision_score(y_true=use_train[target],y_pred=predict_matrix)

0.6199859957987396

3.재현율

In [35]:
confu_mtx[1,1]/(confu_mtx[1,0]+confu_mtx[1,1])

0.9998386836586546

In [36]:
skmet.recall_score(y_true=use_train[target],y_pred=predict_matrix)

0.9998386836586546

4.f1 score : 재현율과 정밀도의 조화평균

In [37]:
skmet.f1_score(y_true=use_train[target],y_pred=predict_matrix)

0.7653741664608544

In [38]:
2/(1/skmet.precision_score(y_true=use_train[target],y_pred=predict_matrix)+1/skmet.recall_score(y_true=use_train[target],y_pred=predict_matrix))

0.7653741664608545